In [1]:
import numpy as np
import sqlite3
import os
from scipy.spatial import cKDTree # type: ignore 
main=0

base_dir = os.getcwd()
DB_PATH = os.path.join(base_dir, "data", "Main.db")

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

cursor.execute('SELECT * FROM BaseParameter WHERE baseValue=(?)', (main, ))
row = cursor.fetchone()

print(row)

single=row[1]
h=row[2]

(0, 4e-07, 7e-07)


In [2]:
N=int(250)*2
r=N*single
x=np.linspace(-(r/2),(r/2),N)

lambda0=0.8e-6
lambda1=0.6e-6

f=60e-6

ftx0 = -(2 * np.pi) / lambda0 * (np.sqrt(x**2 + f**2) - f)
ftx0 = np.mod(ftx0, 2 * np.pi)  # wrapTo2Pi

ftx1 = -(2 * np.pi) / lambda1 * (np.sqrt(x**2 + f**2) - f)
ftx1 = np.mod(ftx1, 2 * np.pi)  # wrapTo2Pi

In [3]:
cursor.execute('SELECT angleIn1, angleIn5 FROM Parameter WHERE baseValue=(?)', (main,))
rows = cursor.fetchall()

points = np.array(rows)  # 形状为 (M, 2)，M是数据库行数
print(points)

query_points = np.column_stack((ftx0, ftx1))

tree = cKDTree(points, leafsize=40)
distances, indices = tree.query(query_points, p=1)

di = np.sum(distances)

[[ 2.5146848  -0.45285366]
 [ 2.53438446 -0.42894523]
 [ 2.55448533 -0.40309526]
 ...
 [-1.30594477  2.62702661]
 [-1.30594477  2.62702661]
 [-1.30594477  2.62702661]]


In [4]:
cursor.execute('SELECT angleIn1, angleIn5 FROM Parameter WHERE baseValue=(?)', (main,))
rows = cursor.fetchall()

# 转换为 numpy 数组并检查数据类型
points = np.array(rows)
print("points dtype:", points.dtype)

# 检查是否含 None（数据库 NULL 会变成 Python 的 None）
for i, row in enumerate(rows):
    if None in row:
        raise ValueError(f"第 {i} 行含有 None 值：{row}")

# 强制转换成 float64，会立刻报错如果有非法值
points = np.array(rows, dtype=np.float64)

# 检查维度是否为二维，列数必须为2
print("points shape:", points.shape)
if points.ndim != 2 or points.shape[1] != 2:
    raise ValueError("points 必须是形状为 (M, 2) 的二维数组")

# 检查数值是否全为有限数
if not np.all(np.isfinite(points)):
    raise ValueError("points 中包含 nan 或 inf")

# 检查是否有空数组
if points.shape[0] == 0:
    raise ValueError("points 数组为空，不能建立 KDTree")

points dtype: float64
points shape: (2738, 2)
